## 歌曲序列建模

![](./pic/word2vec.png)

### 从word2vec到song2vec

我们把歌曲的id序列取出来，类比于分完词后的句子，送到word2vec中去学习一下，看看会有什么效果。

In [1]:
import multiprocessing
import gensim
import sys
from random import shuffle

path = "./data/output/popular/"

def parse_playlist_get_sequence(in_line, playlist_sequence):
    song_sequence = []
    contents = in_line.strip().split("\t")
    # 解析歌单序列
    for song in contents[1:]:
        try:
            song_id, song_name, artist, popularity = song.split(":::")
            song_sequence.append(song_id)
        except:
            print("song format error")
            print(song+"\n")
    for i in range(len(song_sequence)):
        shuffle(song_sequence)
        playlist_sequence.append(song_sequence)


def train_song2vec(in_file, out_file):
    #所有歌单序列
    playlist_sequence = []
    #遍历所有歌单
    for line in open(in_file,encoding='utf-8'):
        parse_playlist_get_sequence(line, playlist_sequence)
    #使用word2vec训练
    cores = multiprocessing.cpu_count()
    print("using all "+str(cores)+" cores")
    print("Training word2vec model...")
    model = gensim.models.Word2Vec(sentences=playlist_sequence, size=150, min_count=3, window=7, workers=cores)
    print("Saving model...")
    model.save(out_file)

In [2]:
song_sequence_file = "./data/popular.playlist"
model_file = "./model/song2vec.model"
train_song2vec(song_sequence_file, model_file)

song format error
405599088:::Make Them Wheels Roll

song format error
:::SAFIA:::100.0

song format error
424496188:::大王叫我来巡山 - （原唱：

song format error
 贾乃亮/贾云馨）:::流浪的蛙蛙:::65.0

song format error
19169096:::

song format error
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

song format error
26902203:::What’s your name? (collaboration with 壇蜜)

song format error
:::SoulJa:::100.0

song format error
33054290:::

song format error
Heartbeats:::Dabin:::95.0

song format error
427373827:::Champions (From "Hands of Stone") 

song format error
:::Usher:::30.0

song format error
31654811:::

song format error
American Cowboys:::Tim Wynn:::65.0

song format error
19169096:::

song format error
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

song format error
19169096:::

song format error
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

using all 4 cores
Training word2vec model...
Saving model...


模型已经训练完了，咱们来试一把预测，看看效果。这个预测的过程，实际上就是对某首歌曲，查找“最近”的歌曲（向量距离最近的歌曲）

In [3]:
import pickle
song_dic = pickle.load(open(path+"popular_song.pkl","rb"))
model = gensim.models.Word2Vec.load(model_file)
for song in list(song_dic.keys())[:10]:
    print(song, song_dic[song])

22679160 シングル・アゲイン	徳永英明
187984 飞	周华健
210905 欢聚	陈淑桦
351475 石头记	达明一派
110139 遥远的天空底下	李健
28768109 一个短篇	腰乐队
154093 烟雨蒙蒙	晓艺
77158 冲动的惩罚	刀郎
109962 往事借过	林志炫
31460281 想把我唱给你听	秋若


In [4]:
song_id_list = list(song_dic.keys())[1000:1500:50]
for song_id in song_id_list:
    result_song_list = model.most_similar(song_id)
    print(song_id, song_dic[song_id])
    print("\n相似歌曲 和 相似度 分别为:")
    for song in result_song_list:
        print("\t", song_dic[song[0]], song[1])
    print("\n")

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
116489 生命之曲	林子祥

相似歌曲 和 相似度 分别为:
	 亚拉伯跳舞女郎	Beyond 0.9960165619850159
	 Home Sick Song	Beyond 0.9956828355789185
	 战无不胜	陈小春 0.9569781422615051
	 真情人	李玟 0.9266257286071777
	 乱世巨星	陈小春 0.9194033741950989
	 老虎 鱼虾蟹 电影《林阿珍-老虎 鱼虾蟹》主题曲	李韵明 0.8786227107048035
	 候补情人	Terry 0.8723733425140381
	 两心依然走近	刘小慧 0.8674477338790894
	 活火山	谢霆锋 0.8549770712852478
	 不爱不散	邓丽欣 0.8390281200408936


288986 龙船调	宋祖英

相似歌曲 和 相似度 分别为:
	 迎春花	李采霞 0.9530176520347595
	 如果再回到从前	崔苔菁 0.8586177825927734
	 流星下的祈祷	李采霞 0.8392005562782288
	 妹妹找哥泪花流	李谷一 0.788612961769104
	 太阳出来喜洋洋(四川民歌)	练正华 0.7860589623451233
	 借我一点爱	童安格 0.708297610282898
	 飞天	含笑 0.6711644530296326
	 《西游记》(86版)片头音乐	许镜清 0.6681008338928223
	 其实你不懂我的心	童安格 0.6381615996360779
	 东京萝莉控	神威鬼鸣 0.5832850

所以我们用word2vec学会了哪些歌曲和哪些歌曲最接近。